In [35]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

In [36]:
import numpy as np
import matplotlib.pyplot as plt

### データの読み込み

In [37]:
# データの読み込み

import pandas as pd
filename_input = 'input.csv'  # size: 750 row * 7 column
data_input = pd.read_csv(
    filename_input, 
    sep = ',',   # 改行の\t ではなく、,にしないと7列に認識しない 
    header = 0 #Header: 1st line
    )
filename_output = 'output.csv'  # size: 750 row * 3 column
data_output_raw = pd.read_csv(
    filename_output, 
    sep = ',',
    header = 0 #Header: 1st line
    )

In [38]:
data_output_raw.head(3)

,MFB50,IMEP,Pmax
0,9.00,8.895135,44.077
1,10.25,8.679991,43.328
2,25.75,7.991108,26.776


Output layer normalization

In [39]:
print("MFB50 max: ", max(data_output_raw.MFB50))
print("MFB50 min: ", min(data_output_raw.MFB50))
print("IMEP max: ", max(data_output_raw.IMEP))
print("IMEP min: ", min(data_output_raw.IMEP))
print("Pmax max: ", max(data_output_raw.Pmax))
print("Pmax min: ", min(data_output_raw.Pmax))
data_output = data_output_raw
data_output.MFB50 = data_output_raw.MFB50 / 50
data_output.IMEP = data_output_raw.IMEP / 20
data_output.Pmax = data_output_raw.Pmax / 100
print(data_output.head(3))
type(data_output)


MFB50 max:  29.75
MFB50 min:  3.0
IMEP max:  9.164684296
IMEP min:  7.814703941
Pmax max:  53.86600000000001
Pmax min:  23.369
   MFB50      IMEP     Pmax
0  0.180  0.444757  0.44077
1  0.205  0.434000  0.43328
2  0.515  0.399555  0.26776


pandas.core.frame.DataFrame

In [40]:
"""
Get information of input and output, use it for NN configuration
1. size_input = columns of data_input
2. size_output = columns of data_ouput
3. size_input_data = rows of data_input
4. size_output_data = rows of data_output
"""
"""
PANDAS BASIC
df.shape     (rows,columns) 

df.idxmin()/df.idxmax()    Minimum/Maximum index value --> might be used for normalization
df.mean()                  Mean of values
df.median()                Median of values
"""
size_input = data_input.shape[1]
size_output = data_output.shape[1]
size_input_data = data_input.shape[0]
size_output_data = data_output.shape[0]
print("size_input: ", size_input, ";  number of data: ", size_input_data)
print("size_output: ", size_output, ";  number of data: ", size_output_data)

size_input:  7 ;  number of data:  750
size_output:  3 ;  number of data:  750


In [41]:
"""
test: how to get pandas.dataframe index name??
"""
data_input_index = data_input.keys()
data_input_index_array = data_input_index.values
data_output_index = data_output.keys()
data_output_index_array = data_output_index.values

In [42]:
"""
Visualization of input and output

Plotly: 
https://plotly.com/python/getting-started-with-chart-studio/
Subplot  https://plotly.com/python/creating-and-updating-figures/
"""

# import plotly.plotly as py
# import plotly.graph_objs as go
import chart_studio
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots


"""
INPUT
"""
fig_input = make_subplots(rows=size_input, cols=1) #Variable: size_input
fig_input.update_layout(title_text="Input (normalized)",
                  title_font_size=30)
for i in range(size_input):
    fig_input.add_trace(go.Scatter(
        y = data_input.iloc[:,i], 
        text = data_input_index_array[i],
        mode='markers', 
        name = data_input_index_array[i]
), row=(i+1),col=1)

"""
OUTPUT
"""
fig_output = make_subplots(rows=size_output, cols=1)
fig_output.update_layout(title_text="Output",
                  title_font_size=30)
for i in range(size_output):
    fig_output.add_trace(go.Scatter(
        y = data_output.iloc[:,i], 
        text = data_output_index_array[i],
        mode='markers', 
        name = data_output_index_array[i]
), row=(i+1),col=1)

#pio.show(fig)
fig_input.show()
fig_output.show()

### データ分割

In [43]:
from sklearn.model_selection import train_test_split
#x_train, x_test, t_train, t_test = train_test_split(input, output, test_size = 0.25)

"""
Configuration:  split ratio of test data
c_test_size
"""
c_test_size = 0.3

x_train_df, x_test_df, t_train_df, t_test_df = train_test_split(data_input, data_output, test_size = c_test_size)
x_train = x_train_df.values  #Data type conversion
x_test = x_test_df.values
t_train = t_train_df.values
t_test = t_test_df.values

In [44]:
print("train data", x_train.shape, ";  test data", x_test.shape)
# x_train.shape[0]  --> to get row numbers
# x_train[1]  --> to get row values

train data (525, 7) ;  test data (225, 7)


### Dropuoutの有無、割り合いの設定

In [45]:
"""
Configuration for dropout usage
"""
lc_use_dropout = True  # Dropoutなしのときの場合はFalseに
c_dropout_ratio = 0.2

### Network設定 MultilayerNetExtend
    input_size   7
    hidden size  7-5-3-1-3-5


In [46]:
"""
Configuration of network
"""
size_hidden_list = [20]
"""
Check Network configuration lables working or not
"""
print("input_size = ", size_input)
print("output_size = ", size_output)
print("hidden_size_list = ", size_hidden_list)
print("use_dropout = ", lc_use_dropout)
print("dropout_ratio = ", c_dropout_ratio)

input_size =  7
output_size =  3
hidden_size_list =  [20]
use_dropout =  True
dropout_ratio =  0.2


In [47]:
from common.multi_layer_net_extend import MultiLayerNetExtend

network = MultiLayerNetExtend(
    input_size=size_input, 
    hidden_size_list=size_hidden_list,            #[7, 5, 3, 3, 3, 5]              
    output_size=size_output, 
    use_dropout = lc_use_dropout, 
    dropout_ration = c_dropout_ratio,
    activation = 'sigmoid',
    weight_init_std='xavier',
    weight_decay_lambda=0.8,
    use_batchnorm=True
)

print("New neural network created successfully: network")
#print(network.activation)

New neural network created successfully: network


In [48]:
"""
Configuration of training process
"""
c_size_epochs = 200
c_size_mini_batch = 25
c_conf_optimizer = 'sgd'
c_conf_optimizer_para = {'lr': 0.01}
lc_verbose = False
"""
Check Network configuration lables working or not
"""
print("size_epochs = ", c_size_epochs, type(c_size_epochs))
print("size_mini_batch = ", c_size_mini_batch, type(c_size_mini_batch))
print("conf_optimizer = ", c_conf_optimizer)
print("conf_optimizer_para = ", c_conf_optimizer_para)
print("use verbose = ", lc_verbose)

size_epochs =  200 <class 'int'>
size_mini_batch =  25 <class 'int'>
conf_optimizer =  sgd
conf_optimizer_para =  {'lr': 0.01}
use verbose =  False


In [49]:
from common.trainer import Trainer

trainer = Trainer(
    network, 
    x_train, 
    t_train, 
    x_test, 
    t_test,
    epochs = c_size_epochs, 
    mini_batch_size = c_size_mini_batch,
    optimizer = c_conf_optimizer, 
    optimizer_param = c_conf_optimizer_para, 
    verbose = lc_verbose
)


In [50]:
print(trainer.train_size)
print(trainer.batch_size)
print(trainer.iter_per_epoch)  #train_size / batch_size needs to be integer
print(type(trainer.iter_per_epoch))
x=int(trainer.iter_per_epoch)
print(type(x))
print(trainer.max_iter)

525
25
21
<class 'int'>
<class 'int'>
4200


In [51]:

trainer.train()
train_acc_list, test_acc_list = trainer.train_acc_list, trainer.test_acc_list

In [52]:
print(trainer.current_iter)
#print(trainer.train_step.x_batch)

4200


### グラフの描画

In [53]:
"""
matplot

markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

"""


'\nmatplot\n\nmarkers = {\'train\': \'o\', \'test\': \'s\'}\nx = np.arange(len(train_acc_list))\nplt.plot(x, train_acc_list, marker=\'o\', label=\'train\', markevery=10)\nplt.plot(x, test_acc_list, marker=\'s\', label=\'test\', markevery=10)\nplt.xlabel("epochs")\nplt.ylabel("accuracy")\nplt.ylim(0, 1.0)\nplt.legend(loc=\'lower right\')\nplt.show()\n\n'

In [54]:
'''
Put accuracy data in pandas data frame
Prepare for plotly express
'''
'''
print(len(train_acc_list))
print(len(test_acc_list))
data = {
    'train_accuracy': train_acc_list,
    'test_accuracy':test_acc_list
}
DF = pd.DataFrame(data, columns = {'train_accuracy', 'test_accuracy'})
print(DF.head())
'''

"\nprint(len(train_acc_list))\nprint(len(test_acc_list))\ndata = {\n    'train_accuracy': train_acc_list,\n    'test_accuracy':test_acc_list\n}\nDF = pd.DataFrame(data, columns = {'train_accuracy', 'test_accuracy'})\nprint(DF.head())\n"

In [55]:
"""
Figure Line with several trace
1. import graph_objects (not objs anymore)
2. Create a blank figure 
3. Add trace as you want
4. Update the layout
5. Show figure
"""

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(len(train_acc_list)),
    y=train_acc_list,
    name='train',
    line=dict(color='firebrick', width=2)
    ))
fig.add_trace(go.Scatter(
    x=np.arange(len(train_acc_list)),
    y=test_acc_list,
    name='test',
    line=dict(color='royalblue', width=2)
    ))

fig.update_layout(title='Comparison of Train Accuracy and Test Accuracy',
                   xaxis_title='Epochs',
                   yaxis_title='Accuracy (0~1)')

fig.show()

In [56]:
x_test[1], t_test[1]
x_T = x_test[1]
y = t_test[1]
x = x_T.reshape(1,-1)
np.shape(x)
x.dtype
network.predict(x,train_flg = True)
tgt = t_test[1]
pdt = network.predict(x)
print(tgt, pdt)

[0.125      0.44514279 0.4876    ] [[0.20709844 0.43830456 0.43586695]]


In [57]:
"""
data_output.MFB50 = data_output_raw.MFB50 / 50
data_output.IMEP = data_output_raw.IMEP / 20
data_output.Pmax = data_output_raw.Pmax / 100
"""

pdt_raw=network.predict(x_test)
pdt=pdt_raw
pdt[:,0] = pdt_raw[:,0]*50
pdt[:,1] = pdt_raw[:,1]*20
pdt[:,2] = pdt_raw[:,2]*100
tgt=t_test
tgt[:,0]=t_test[:,0]*50
tgt[:,1]=t_test[:,1]*20
tgt[:,2]=t_test[:,2]*100

print(np.max(t_test[:,2]))
print(np.max(tgt[:,2]))

53.866
53.866


In [58]:
import chart_studio
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

fig_acc = make_subplots(rows=1, cols=pdt.shape[1]) #Variable: size_input
fig_acc.update_layout(title_text="Correlation between predicted value and training data",
                  title_font_size=30)
for i in range(pdt.shape[1]):
    fig_acc.add_trace(go.Scatter(
        x=pdt[:,i],
        y=tgt[:,i], 
        text = data_output_index_array[i],
        mode='markers', 
        name = data_output_index_array[i]
), row=1,col=(i+1))
    fig_acc.add_trace(go.Scatter(
        x=pdt[:,i],
        y=tgt[:,i], 
        text = data_output_index_array[i],
        mode='markers', 
        name = data_output_index_array[i]
), row=1,col=(i+1))

In [59]:
fig_acc.show()

In [60]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [61]:
#fig.write_image("images/fig_acc.png")
fig.write_image("images/fig{0}.jpeg".format("1"))
fig_acc.write_image("images/fig{0}.jpeg".format("2"))
#fig.write_image("images/fig_acc.webp")
#fig.write_image("images/fig_acc.svg")
#fig.write_image("images/fig_acc.pdf")
#fig.write_image("images/fig_acc.eps")


In [62]:
def r2_func(x,y):
    # scikit-learnの linear_modelを読み込み
    from sklearn import linear_model
    #インスタンス作成
    clf = linear_model.LinearRegression()
    # 単回帰モデルを作成
    clf.fit(x,y)
    # y=ax+bのa
    a = clf.coef_
    # y=ax+bのb
    b = clf.intercept_
    #yの標本分散
    sy2 = y.var(ddof=False)
    #線形回帰式との誤差を計算
    d = y-a*x-b
    #誤差の2乗平均を計算
    syx2 = np.mean(d**2)
    #sr^2を計算
    sr2 = sy2 - syx2
    #決定係数計算
    r2 = sr2/sy2
    
    return r2

In [63]:
print(pdt.shape, tgt.shape)
from sklearn.metrics import r2_score
r2 = np.zeros(3)
for i in range(pdt.shape[1]):
    r2[i]=r2_score(pdt[:,i],tgt[:,i])
print(r2)

(225, 3) (225, 3)
[-3.19413866 -6.19179528 -3.48889218]


In [64]:
"""
How to show network parameters?

network.params
network.params["W1"]

"""


'\nHow to show network parameters?\n\nnetwork.params\nnetwork.params["W1"]\n\n'